In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

# <a href="https://en.wikipedia.org/wiki/Unification_(computer_science)">Unification</a>

This notebook implements the algorithm of *Martelli and Montanari* for the unification of terms.

## Utility Functions

In [ ]:
import { Tuple, Value } from "recursive-set";
import type { Term, Signature } from "./FOL-Parser";
import { parseTerm as parseTermFromParser, createVarTerm, createFunTerm } from "./FOL-Parser";

Formulas are represented as nested arrays. In order to convert a string into a nested array we use the class `LogicParser` that is implemented in the module `FOL-Parser`. Our parser distinguishes variables and function symbols as follows:
- A word starting with a lower case letter is interpreted as a *variable*.
- A word starting with an upper case letter is assumed to be a *function* or *predicate symbol*.

In [ ]:
function parseTerm(s: string, signature: Signature): Term {
  return parseTermFromParser(s, signature);
}

In [ ]:
const demoSig: Signature = {
  functions: new Map([['F', 2], ['G', 1]]),
  predicates: new Map()
};

parseTerm('F(G(x),y)', demoSig);

The method $\texttt{apply}(t, \sigma)$ takes an object $t$ and a substitution $\sigma$ and computes $t\sigma$, i.e. it *applies* the substitution $\sigma$ to $t$. The object $t$ is either a term, a *syntactic equation*, or a `Set` of syntactic equations. The substitution $\sigma$ is represented as a JavaScript object (Record). Assume that $\sigma = \bigl\{ x_1 \mapsto t_1, \cdots, x_n \mapsto t_n \bigr\}$. Then $t\sigma$ is defined by induction on $t$:
- If $t$ is a variable, there are two cases when defining $t\sigma$:
  - $t = x_i$ for an $i\in\{1,\cdots,n\}$. Then we define 
    $$ x_i\sigma := t_i. $$
  - $t = y$ where $y\in\mathcal{V}$, but $y \not\in \{x_1,\cdots,x_n\}$. Then we define 
    $$ y\sigma := y.$$
- Otherwise, we must have $t = f(s_1,\cdots,s_m)$. Then we define: 
  $$ f(s_1, \cdots, s_m)\sigma := f(s_1\sigma, \cdots, s_m\sigma). $$

In [ ]:
// Hilfsfunktionen für Tuple-Zugriff (kompatibel mit recursive-set)
function getFirst(t: Tuple<Value[]>): string {
  const value = t.get(0);
  if (typeof value !== "string") {
    throw new Error("Erstes Element des Tuples muss ein String sein");
  }
  return value;
}

function isVarTerm(t: Term): boolean {
  return t.length === 1;
}

function isFunTerm(t: Term): boolean {
  return t.length === 2;
}

function getArgs(t: Term): Tuple<Value[]> {
  if (t.length !== 2) {
    throw new Error("Nur Funktions-Terme haben Argumente");
  }
  const args = t.get(1);
  if (!(args instanceof Tuple)) {
    throw new Error("Zweites Element muss ein Tuple sein");
  }
  return args as Tuple<Value[]>;
}


In [ ]:
type Substitution = Map<string, Term>;

function apply(t: Term, σ: Substitution): Term {
  // Fall 1: Variable (VarTerm - Tuple mit Länge 1)
  if (isVarTerm(t)) {
    const varName = getFirst(t);
    const substituted = σ.get(varName);
    if (substituted !== undefined) {
      return substituted;
    }
    return t;
  }

  // Fall 2: Funktions-Term (FunTerm - Tuple mit Länge 2)
  if (isFunTerm(t)) {
    const funSymbol = getFirst(t);
    const args = getArgs(t);
    
    // Rekursiv auf alle Argumente anwenden
    const newArgs: Term[] = [];
    for (let i = 0; i < args.length; i++) {
      const arg = args.get(i);
      if (arg instanceof Tuple) {
        newArgs.push(apply(arg, σ));
      } else {
        throw new Error("Argument muss ein Term (Tuple) sein");
      }
    }
    
    return createFunTerm(funSymbol, newArgs);
  }

  return t;
}

In [ ]:
const sig1: Signature = {
  functions: new Map([['G', 1], ['H', 2]]),
  predicates: new Map()
};

const s1 = parseTerm('G(z)', sig1);
const s2 = parseTerm('H(u, v)', sig1);
const σ: Substitution = new Map<string, Term>([
  ['x', s1],
  ['y', s2]
]);
σ

In [ ]:
const sig2: Signature = {
  functions: new Map([['F', 3], ['G', 1], ['H', 2]]),
  predicates: new Map()
};

const t = parseTerm('F(x,H(y,x),G(z))', sig2);
t

In [ ]:
apply(t, σ);

If  $\sigma = \big\{ x_1 \mapsto s_1, \cdots, x_m \mapsto s_m \big\}$ and
$\tau = \big\{ y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big\}$ 
are two substitutions that are <em style="color:blue;">non-overlapping</em>, i.e. such that $\texttt{dom}(\sigma) \cap \texttt{dom}(\tau) = \{\}$ holds,
then we define the <em style="color:blue;">composition</em> $\sigma\tau$ of $\sigma$ and $\tau$ as follows:
$$\sigma\tau := \big\{ x_1 \mapsto s_1\tau, \cdots, x_m \mapsto s_m\tau,\; y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big\}$$
The function $\texttt{compose}(\sigma, \tau)$ takes two non-overlapping substitutions and computes the composition $\sigma\tau$.

In [ ]:
function compose(σ: Substitution, τ: Substitution): Substitution {
  const result = new Map<string, Term>(τ);
  
  for (const [x, s] of σ.entries()) {
    result.set(x, apply(s, τ));
  }
  
  return result;
}

In [ ]:
const τ: Substitution = new Map<string, Term>([
  ['z', s1],
  ['u', s2]
]);
τ

In [ ]:
σ

In [ ]:
compose(σ, τ);

The function $\texttt{occurs}(x, t)$ checks whether the variable $x$ occurs in the term $t$.

In [ ]:
function occurs(x: string, t: Term): boolean {
  // Variable
  if (isVarTerm(t)) {
    return getFirst(t) === x;
  }
  
  // Funktions-Term
  if (isFunTerm(t)) {
    const args = getArgs(t);
    for (let i = 0; i < args.length; i++) {
      const arg = args.get(i);
      if (arg instanceof Tuple && occurs(x, arg)) {
        return true;
      }
    }
  }
  
  return false;
}

In [ ]:
t

In [ ]:
occurs('u', t);

In [ ]:
occurs('x', t);

## The Algorithm of Martelli and Montanari

The rules of Martelli and Montanari that can be used to solve a system of syntactical equations are as follows:
<ol>
<li> If $y\in\mathcal{V}$ is a variable that does <b style="color:red;">not</b> occur in the term $t$,
     then we perform the following reduction: 
     $$ \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle \quad\leadsto \quad 
         \Big\langle E\{y \mapsto t\}, \sigma\big\{ y \mapsto t \big\} \Big\rangle 
     $$
</li>      
<li> If the variable $y$ occurs in the term $t$, then the system of syntactical equations
     $E \cup \big\{ y \doteq t \big\}$ is not solvable:
     $$ \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle\;\leadsto\; \texttt{None} \quad
        \mbox{if $y \in \textrm{Var}(t)$ and $y \not=t$.}$$
</li>
<li> If $y\in\mathcal{V}$ is a variable and $t$ is no variable, then we use the following rule:
     $$ \Big\langle E \cup \big\{ t \doteq y \big\}, \sigma \Big\rangle \quad\leadsto \quad 
         \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle.
     $$   
</li>
<li> Trivial syntactical equations of variables can be dropped:
     $$ \Big\langle E \cup \big\{ x \doteq x \big\}, \sigma \Big\rangle \quad\leadsto \quad
         \Big\langle E, \sigma \Big\rangle.
     $$   
</li>
<li> If $f$ is an $n$-ary function symbol, then we have: 
     $$ \Big\langle E \cup \big\{ f(s_1,\cdots,s_n) \doteq f(t_1,\cdots,t_n) \big\}, \sigma \Big\rangle 
         \;\leadsto\; 
         \Big\langle E \cup \big\{ s_1 \doteq t_1, \cdots, s_n \doteq t_n\}, \sigma \Big\rangle.
     $$   
</li>
<li> The system of syntactical equations $E \cup \big\{ f(s_1,\cdots,s_m) \doteq g(t_1,\cdots,t_n) \big\}$
     has <b style="color:red;">no</b> solution if the function symbols $f$ and $g$ are different:
     $$ \Big\langle E \cup \big\{ f(s_1,\cdots,s_m) \doteq g(t_1,\cdots,t_n) \big\},
      \sigma \Big\rangle \;\leadsto\; \texttt{None} \qquad \mbox{if $f \not= g$}.
     $$
</ol>


In [ ]:
type Equation = ['≐', Term, Term];

function termsEqual(s: Term, t: Term): boolean {
  // Beide Variablen
  if (isVarTerm(s) && isVarTerm(t)) {
    return getFirst(s) === getFirst(t);
  }
  
  // Beide Funktionen
  if (isFunTerm(s) && isFunTerm(t)) {
    const sSymbol = getFirst(s);
    const tSymbol = getFirst(t);
    
    if (sSymbol !== tSymbol) return false;
    
    const sArgs = getArgs(s);
    const tArgs = getArgs(t);
    
    if (sArgs.length !== tArgs.length) return false;
    
    for (let i = 0; i < sArgs.length; i++) {
      const sArg = sArgs.get(i);
      const tArg = tArgs.get(i);
      if (sArg instanceof Tuple && tArg instanceof Tuple) {
        if (!termsEqual(sArg, tArg)) return false;
      } else {
        return false;
      }
    }
    
    return true;
  }
  
  return false;
}

function applyToEquations(E: Set<Equation>, σ: Substitution): Set<Equation> {
  const newSet = new Set<Equation>();
  for (const eq of E) {
    const [op, s, t] = eq;
    newSet.add([op, apply(s, σ), apply(t, σ)]);
  }
  return newSet;
}


Given a set of <em style="color:blue;">syntactical equations</em> $E$ and a substitution $\sigma$, the function $\texttt{solve}(E, \sigma)$ uses the rules of Martelli and Montanari to solve $E$.

In [ ]:
function solve(E: Set<Equation>, σ: Substitution): Substitution | null {
  while (E.size > 0) {
    const iter = E.values().next();
    if (iter.done) break; // Sollte nie passieren, aber TypeScript ist happy
    
    const equation = iter.value;
    E.delete(equation);
    
    const [_, s, t] = equation;
    
    // 1. Remove trivial equations: s ≐ s
    if (termsEqual(s, t)) {
      continue;
    }
    
    // 2. Variable Elimination: x ≐ t
    if (isVarTerm(s)) {
      const varName = getFirst(s);
      
      if (occurs(varName, t)) {
        return null; // Failure: Occurs Check
      } else {
        const sub = new Map([[varName, t]]);
        // Wende {x -> t} auf die restlichen Gleichungen in E an
        E = applyToEquations(E, sub);
        // Komposition der Substitution
        σ = compose(σ, sub);
      }
    }
    // 3. Orientation: t ≐ x -> x ≐ t
    else if (isVarTerm(t)) {
      E.add(['≐', t, s]);
    }
    // 4. Decomposition: f(...) ≐ g(...)
    else if (isFunTerm(s) && isFunTerm(t)) {
      const f = getFirst(s);
      const g = getFirst(t);
      
      const sArgs = getArgs(s);
      const tArgs = getArgs(t);
      
      const m = sArgs.length;
      const n = tArgs.length;
      
      if (f !== g || m !== n) {
        return null; // Failure: Clash (different functors or arity)
      } else {
        for (let i = 0; i < m; i++) {
          const sArg = sArgs.get(i);
          const tArg = tArgs.get(i);
          if (sArg instanceof Tuple && tArg instanceof Tuple) {
            E.add(['≐', sArg, tArg]);
          }
        }
      }
    } else {
      return null;
    }
  }
  
  return σ;
}

Given two terms $s$ and $t$, the function $\texttt{unify}(s, t)$ computes the <em style="color:blue;">most general unifier</em> of $s$ and $t$.

In [ ]:
function unify(s: Term, t: Term): Substitution | null {
  const initialEquation: Equation = ['≐', s, t];
  return solve(new Set<Equation>([initialEquation]), new Map<string, Term>());
}

In [ ]:
const unifyTestSig: Signature = {
  functions: new Map([
    ['P', 2],
    ['F', 1],
    ['G', 1]
  ]),
  predicates: new Map()
};

const t1 = parseTerm('P(x1,F(x1))', unifyTestSig);
const t2 = parseTerm('P(G(x2),x3)', unifyTestSig);
[t1, t2];

In [ ]:
const μ = unify(t1, t2);
μ;